In [1]:
import warnings
from traffic.data import opensky        # Cargamos opensky para poder descargar los datos
from traffic.core import Traffic        # Cargamos la función Traffic de los tipos de aeronaves
from datetime import date, timedelta    # Cargamos el fomrato de tiempo para poder trabajar con él
import pandas as pd                     # Cargamos la libreria pandas para poder trabajar con ella

warnings.filterwarnings("ignore")       # esto nos permite elimnar los avisos

C:\Users\javie\anaconda3\lib\site-packages\pkg_resources\__init__.py:1152: DeprecationWarning: Use of .. or absolute path in a resource path is not allowed and will raise exceptions in a future release.
  self, resource_name
C:\Users\javie\anaconda3\lib\site-packages\ipyleaflet\leaflet.py:338: DeprecationWarning: metadata {'dtype': None} was set from the constructor. With traitlets 4.1, metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  data = Dataset().tag(dtype=None, sync=True, to_json=ds_x_to_json)


In [2]:
# Cargamos la BBDD con la que queremos trabajar. Esta BBDD está en formato csv y debe estar en la misma carpeta que este script
# Ahora tenemos que com0probar que el csv es válido y carga los datos con la información y el dtype correspondiente
Bilbao = pd.read_csv("BBDD_Bilbao_prueba.csv")     #Cargamos en DataFrame

# Eliminamos de la BBDD la columna Unnamed
Bilbao = Bilbao.drop("Unnamed: 0", axis = 1)

Bilbao.info()
# Podemos comprobar que ya no tenemos problemas de formato y de que tenemos todo en float64 e int64 y no tenemos la columna de Unnamed

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5852 entries, 0 to 5851
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   altitude_1          5852 non-null   float64
 1   geoaltitude_1       5852 non-null   float64
 2   latitude_1          5852 non-null   float64
 3   longitude_1         5852 non-null   float64
 4   cluster_1           5852 non-null   int64  
 5   cluster_2           5852 non-null   int64  
 6   Init separation     5852 non-null   float64
 7   Init acimut         5852 non-null   float64
 8   Var GS Module       5852 non-null   float64
 9   Var Track           5852 non-null   float64
 10  Var init altitude   5852 non-null   float64
 11  Var Vertical speed  5852 non-null   float64
 12  Conflicto           5852 non-null   int64  
dtypes: float64(10), int64(3)
memory usage: 594.5 KB


# 1. Conflictos entre los flujos

In [4]:
# Vamos a determinar losflujos de tráfico en los que se producen cruces
con = Bilbao.loc[(Bilbao['Conflicto'] == 1)]
columnas = ['cluster_1', 'cluster_2']
# Podemos obtener los flujos que han presentado algún conflicto, se trata de valores únicos por lo que podríamos trabajar solo con estos
flujos_con_cruce = con[columnas].drop_duplicates()
# Por ejemplo si quiero trabajar con un flujo en particular tengo que seleccionar los valorers de una columna
Flujo_1 = flujos_con_cruce.iloc[0]
# Donde Flujo_1 es el vector que contiene los flujos a considerar y Flujo_1['cluster_1'] y Flujo_2['cluster_2'] nos indica a que flujos pertenecen

In [5]:
# El siguiente paso es reducir la BBDD de todos los vuelos a solamente aquellos que coinciden con un par de flujos especifícos
BBDD_Flujos = Bilbao.loc[Bilbao['cluster_1'] == Flujo_1['cluster_1']]
BBDD_Flujos = BBDD_Flujos.loc[Bilbao['cluster_2'] == Flujo_1['cluster_2']]

2       0
9       0
10      0
20      0
22      0
       ..
5747    0
5749    0
5758    0
5760    0
5763    0
Name: cluster_2, Length: 768, dtype: int64

Si no el siguiente paso sería obtener una matriz para cada uno de los flujos conflictos para que se automatice
Aquí lo estamos calculando para las aeronaves del flujo 1 con el 0 por ejemplo, pero del 0 al 1 es otro flujo.  
```
j = 1
for i, flow in flujos_con_cruce.iterrows():
    # Seleccionamos de la BBDD las filas que cumplen el cluster_1
    filas_1 = BBDD_2.loc[BBDD_2['cluster_1'] == flow['cluster_1']]
    # Seleccionamos del primer filtrado las filas que cumplen el cluster_2
    filas_2 = filas_1.loc[filas_1['cluster_2'] == flow['cluster_2']]
    # Le damos un nombre con un número distinto a cada una de las matrices
    exec('Flujos{} = filas_2'.format(j))
    j = j + 1
``` 
Con este paso ya hemos conseguido programar la subdivisión de los flujos para que lo haga de golpe y nos saquen tantas matrices como necesitamos

In [61]:
# Podemos comprobar que no hay ningun vuelo duplicado
BBDD_Flujos[BBDD_Flujos.duplicated(keep=False)]

,altitude_1,geoaltitude_1,latitude_1,longitude_1,cluster_1,cluster_2,Init separation,Init acimut,Var GS Module,Var Track,Var init altitude,Var Vertical speed,Conflicto


In [63]:
# Además podemos extraer información estadística de la BBDD
BBDD_Flujos.describe()

,altitude_1,geoaltitude_1,latitude_1,longitude_1,cluster_1,cluster_2,Init separation,Init acimut,Var GS Module,Var Track,Var init altitude,Var Vertical speed,Conflicto
count,768.000000,768.000000,768.000000,768.000000,768.0,768.0,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,36472.395833,36074.479167,43.058395,-2.904213,-1.0,0.0,9.590656,-40.372238,199.800944,109.398639,1172.395833,-74.666667,0.003906
std,2505.609038,2549.952013,0.126834,0.198640,0.0,0.0,9.016226,108.005006,247.605338,144.624876,2889.256454,376.462604,0.062418
min,31975.000000,31550.000000,42.979028,-3.172302,-1.0,0.0,0.026155,-179.350644,0.999999,-14.571525,-5400.000000,-960.000000,0.000000
25%,35987.500000,35350.000000,42.982445,-3.017431,-1.0,0.0,2.849829,-105.677328,42.746305,-1.091538,-975.000000,-128.000000,0.000000
50%,36000.000000,35600.000000,42.987873,-2.922396,-1.0,0.0,5.798042,-90.788564,71.449084,5.112138,800.000000,0.000000,0.000000
75%,38000.000000,37525.000000,43.058201,-2.841866,-1.0,0.0,17.392139,89.189837,218.748900,289.245024,2050.000000,64.000000,0.000000
max,43000.000000,42750.000000,43.408539,-2.463045,-1.0,0.0,30.904373,179.208438,865.591535,356.414344,11000.000000,1088.000000,1.000000


In [51]:
# ¿Cuantos conflictos hay entre todas las posibilidades?
print(len(BBDD_Flujos), len(BBDD_Flujos[BBDD_Flujos['Conflicto'] == 1]))

# También podemos ver cuantos conflictos tienen entre estos flujos
BBDD_Flujos[BBDD_Flujos['Conflicto'] == 1]

768 3


,altitude_1,geoaltitude_1,latitude_1,longitude_1,cluster_1,cluster_2,Init separation,Init acimut,Var GS Module,Var Track,Var init altitude,Var Vertical speed,Conflicto
62,37625.0,37350.0,42.979028,-2.944336,-1,0,1.490296,-81.532121,35.227799,3.559972,250.0,64.0,1
478,39225.0,38625.0,42.982473,-2.974091,-1,0,2.058785,-92.463909,70.035645,-7.430207,2050.0,192.0,1
3077,36000.0,35700.0,43.020087,-3.024183,-1,0,2.287163,-179.350644,54.817833,0.247659,0.0,0.0,1


In [19]:
# Ahora sobre esta BBDD ya deberíamos poder aplicar los algoritmos de ML
# Los traiing son todos menos las labels (Conflictos)
column = ['cluster_1', 'cluster_2', 'Conflicto']
X = BBDD_Flujos.drop(column, axis = 1)
# Las labels son la columna de conflictos
y = BBDD_Flujos['Conflicto']
# Mostramos la salida de la X
X.head()

,altitude_1,geoaltitude_1,latitude_1,longitude_1,Init separation,Init acimut,Var GS Module,Var Track,Var init altitude,Var Vertical speed
2,37625.0,37350.0,42.979028,-2.944336,1.356611,-90.343315,43.416549,3.946407,1600.0,512.0
9,37625.0,37350.0,42.979028,-2.944336,1.446362,-88.703679,32.649627,3.625368,625.0,-256.0
10,37625.0,37350.0,42.979028,-2.944336,0.942923,-79.585217,51.400345,5.125962,1625.0,448.0
20,37625.0,37350.0,42.979028,-2.944336,0.462716,-79.565758,69.426160,6.614749,3625.0,448.0
22,37625.0,37350.0,42.979028,-2.944336,3.369445,-87.974453,26.925801,-3.360991,450.0,-384.0


# 2. Aplicación de ML

In [25]:
# Podemos comprobar que las label también son 0 y 1 y las posiciones de los conflictos que coinciden con los de la BBDD
y[y == 1]

62      1
478     1
3077    1
Name: Conflicto, dtype: int64

In [26]:
# Dividimos la muestra entre datos de training y de testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X , y , stratify = y, test_size = 0.2, random_state = 42)

# De esta forma tenemos dividia la muestra en un 20% test y 80% train
# Lo he comprobado y he visto que los test X e Y siguen en la misma posición, eso no cambia así que bien, están relacionados los datos con la y

In [35]:
# En este caso podemos ver como han quedado la distribución para las labels en función de que conflictos en que zona
len(y), len(y_train), len(y_test), len(y_train[y_train == 1]), len(y_test[y_test == 1])

(768, 614, 154, 2, 1)

In [36]:
# Vamos a probar con un SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

svm_clf = Pipeline((
    ('scaler', StandardScaler()),
    ('linear_svc', LinearSVC(C=1, loss = 'hinge'))
))

svm_clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linear_svc',
                 LinearSVC(C=1, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='hinge', max_iter=1000, multi_class='ovr',
                           penalty='l2', random_state=None, tol=0.0001,
                           verbose=0))],
         verbose=False)

In [37]:
from sklearn.model_selection import cross_val_score
scores = -1 * cross_val_score(svm_clf, X, y,
                             cv = 5,
                             scoring = 'neg_mean_absolute_error')

In [38]:
scores

array([0.00649351, 0.00649351, 0.00649351, 0.        , 0.        ])

In [39]:
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(svm_clf, X_train, y_train, cv = 3)

In [44]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_train, y_train_pred)

array([[612,   0],
       [  2,   0]], dtype=int64)